<div id="bg" style="height:150px;width:150px; float=left; clear=true; margin-left:35%">
  <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/ce/IE_University_logo.svg/1200px-IE_University_logo.svg.png" style="width:100%; height:100%;">
</div> 
 
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f3/Apache_Spark_logo.svg/1280px-Apache_Spark_logo.svg.png" style="width:10%; height:10%; margin-left:37%">

## <span style="color:black; margin-left:26%; font-size:16pt;">Professor: Asier Abreu Aramburu</span>
## <span style="color:red; margin-left:15%; font-size:22pt;"><u>Challenge #1: Network Intrusion Detection</u></span>
</br>
</br>
</br>

## Team Members:
    1. Resham Gala
    2. Mike Tondu
    3. Mustafa Demir
    4. Nacho
    5. Dan Ryan
    6. Sina
    7. Hani Eid

### Importing libraries and setting dataset path:

In [1]:
import os
my_home=os.environ['HOME']
dataset_path="/home/ubuntu/spark-course/FinalProject/challenge_1/"
outputs_path=my_home
#import findspark
#findspark.init()
#i have commented this since i ran this locally on the vm, if cluster is working for you then uncomment this

In [2]:
import pyspark #for the vm only
from pyspark.sql.functions import *
from pyspark.sql import SQLContext
from pyspark.sql import SparkSessiont
from pyspark.sql.types import *
import pandas as pd

In [3]:

spark = SparkSession \
    .builder \
    .master("local[1]") \
    .appName("GroupProject-TeamE-ML") \
    .config("spark.driver.maxResultSize", "2g") \
    .getOrCreate()

### Loading Data:
    1. full.data
    2. Corrected

In [4]:
df = spark.read \
    .option("inferSchema", "true") \
    .csv("file://"+dataset_path+"full.data")

In [5]:
df_test_1 = spark.read \
    .option("inferSchema", "true") \
    .csv("file://"+dataset_path+"corrected")

### Changing column names based on names text file:

In [6]:
features=[  "duration","protocol_type","service", \
            "flag","src_bytes","dst_bytes", \
            "land","wrong_fragment","urgent", \
            "hot","num_failed_logins","logged_in", \
            "num_compromised","root_shell","su_attempted", \
            "num_root","num_file","num_shells","num_access_files", \
            "num_outbound_cmds","is_host_login","is_guest_login", \
            "count","srv_count","serror_rate","srv_serror_rate", \
            "rerror_rate","srv_rerror_rate","same_srv_rate", \
            "diff_srv_rate","srv_diff_host_rate","dst_host_count", \
            "dst_host_srv_count","dst_host_same_srv_rate","dst_host_diff_srv_rate", \
            "dst_host_same_src_port_rate","dst_host_srv_diff_host_rate", \
            "dst_host_serror_rate","dst_host_srv_serror_rate","dst_host_rerror_rate", \
            "dst_host_srv_rerror_rate","connection_type"]

rawnames=df.schema.names

# Create a small function
def updateColNames(df,oldnames,newnames):
    for i in range(len(newnames)):
        df=df.withColumnRenamed(oldnames[i], newnames[i])
    return df

df=updateColNames(df,rawnames,features)

In [7]:
df_test = updateColNames(df_test_1,rawnames,features)

### Data Inspection

In [8]:
# Print the current schema of the dataset
df.printSchema()

root
 |-- duration: integer (nullable = true)
 |-- protocol_type: string (nullable = true)
 |-- service: string (nullable = true)
 |-- flag: string (nullable = true)
 |-- src_bytes: integer (nullable = true)
 |-- dst_bytes: integer (nullable = true)
 |-- land: integer (nullable = true)
 |-- wrong_fragment: integer (nullable = true)
 |-- urgent: integer (nullable = true)
 |-- hot: integer (nullable = true)
 |-- num_failed_logins: integer (nullable = true)
 |-- logged_in: integer (nullable = true)
 |-- num_compromised: integer (nullable = true)
 |-- root_shell: integer (nullable = true)
 |-- su_attempted: integer (nullable = true)
 |-- num_root: integer (nullable = true)
 |-- num_file: integer (nullable = true)
 |-- num_shells: integer (nullable = true)
 |-- num_access_files: integer (nullable = true)
 |-- num_outbound_cmds: integer (nullable = true)
 |-- is_host_login: integer (nullable = true)
 |-- is_guest_login: integer (nullable = true)
 |-- count: integer (nullable = true)
 |-- srv

In [9]:
df.select('service').distinct().count()

70

In [10]:
df.select('service').distinct().collect()

[Row(service='telnet'),
 Row(service='ftp'),
 Row(service='auth'),
 Row(service='iso_tsap'),
 Row(service='systat'),
 Row(service='name'),
 Row(service='sql_net'),
 Row(service='ntp_u'),
 Row(service='X11'),
 Row(service='pop_3'),
 Row(service='ldap'),
 Row(service='discard'),
 Row(service='tftp_u'),
 Row(service='Z39_50'),
 Row(service='daytime'),
 Row(service='domain_u'),
 Row(service='login'),
 Row(service='smtp'),
 Row(service='http_2784'),
 Row(service='mtp'),
 Row(service='domain'),
 Row(service='http'),
 Row(service='harvest'),
 Row(service='link'),
 Row(service='courier'),
 Row(service='kshell'),
 Row(service='pop_2'),
 Row(service='other'),
 Row(service='exec'),
 Row(service='nnsp'),
 Row(service='efs'),
 Row(service='IRC'),
 Row(service='pm_dump'),
 Row(service='private'),
 Row(service='urh_i'),
 Row(service='ftp_data'),
 Row(service='whois'),
 Row(service='nntp'),
 Row(service='netbios_ns'),
 Row(service='aol'),
 Row(service='klogin'),
 Row(service='shell'),
 Row(service='re

In [ ]:
df.select('protocol_type').distinct().collect()

In [ ]:
df.select('connection_type').distinct().collect()

In [15]:
categoricalColumns = ["protocol_type", "service", "flag"] 

#### We check for level of all categorical variables in both training and test data sets

In [16]:
for col in categoricalColumns:
    print("[", col,"] :", df.select(col).distinct().count())    

[ protocol_type ] : 3
[ service ] : 70
[ flag ] : 11


In [17]:
for col in categoricalColumns:
    print("[",col,"] :",df_test.select(col).distinct().count())    

[ protocol_type ] : 3
[ service ] : 65
[ flag ] : 11


#### We observe that the 'service' variable has more classes in the training set, so thats why we merge them together for data transformation

In [ ]:
print('Nb. of records  : %d' % df.count())

### Data Preprocessing:

In [18]:
#defining the target variable
from pyspark.sql.functions import *

df = df.withColumn('label',when(df.connection_type == 'normal.', 0).otherwise(1))
df_test = df_test.withColumn('label',when(df_test.connection_type == 'normal.',0).otherwise(1))

#### Checking correlations

In [19]:
from pyspark.ml.stat import Correlation
def computeCorrelation(df,targetColumnName):
    corDF = pd.DataFrame(columns=['Target','Ind','Value'])
    ind = 0
    for col in df.columns:
        r=df.stat.corr(col,targetColumnName)
        x = [targetColumnName, col,r]
        corDF = corDF.set_value(ind, corDF.columns, x)
        ind = ind + 1
        #print("Pearson correlation : %s %s %f \n" %(col,targetColumnName,r))
    
    return corDF

In [20]:
numericDf = df.drop('service', 'protocol_type', 'flag', 'connection_type')

In [21]:
corMatrix = computeCorrelation(numericDf, 'label')

In [22]:
corMatrix.sort_values(by=['Value'],  ascending=False)

,Target,Ind,Value
38,label,label,1
19,label,count,0.767425
28,label,dst_host_count,0.656897
20,label,srv_count,0.575426
32,label,dst_host_same_src_port_rate,0.486541
35,label,dst_host_srv_serror_rate,0.230333
21,label,serror_rate,0.230061
22,label,srv_serror_rate,0.229606
34,label,dst_host_serror_rate,0.229516
26,label,diff_srv_rate,0.0159138


#### 1. Creating a new data frame containing both training and test for later transformations

In [23]:
mixedDS = df.union(df_test)

In [25]:
mixedDS.select('protocol_type', 'service', 'dst_bytes', 'connection_type', 'label').limit(10).toPandas()

,protocol_type,service,dst_bytes,connection_type,label
0,tcp,http,45076,normal.,0
1,tcp,http,4528,normal.,0
2,tcp,http,1228,normal.,0
3,tcp,http,2032,normal.,0
4,tcp,http,486,normal.,0
5,tcp,http,1282,normal.,0
6,tcp,http,1337,normal.,0
7,tcp,http,1364,normal.,0
8,tcp,http,1295,normal.,0
9,tcp,http,5450,normal.,0


#### 2. Forming the pipeline stages

In [26]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

stages = [] # stages in our Pipeline
for col in categoricalColumns:
  
  # Category Indexing with StringIndexer
  indexer = StringIndexer(inputCol=col, outputCol=col+"_index")
  #indexer.setHandleInvalid("skip")
    
  # Use OneHotEncoder to convert categorical variables into binary SparseVectors
  encoder = OneHotEncoder(inputCol=col+"_index", outputCol=col+"_vector")
 
  # Add stages.  These are not run here, but will run all at once later on.
  stages += [indexer, encoder]

#### 3. Defining different data types

In [27]:
target = ["label"]
numericCols = ['count', 'dst_host_count', 'srv_count', 'dst_host_same_src_port_rate', 'logged_in']
#numericCols = ["duration", "src_bytes", "dst_bytes", "wrong_fragment", "urgent", "hot", "num_failed_logins", "num_compromised", "num_root", "num_file","num_shells","num_access_files","num_outbound_cmds","count","serror_rate","rerror_rate","same_srv_rate","diff_srv_rate","srv_count","srv_serror_rate","srv_rerror_rate","srv_diff_host_rate"] # you can add extra features if you want, i have not considered all
binaryCols = ["logged_in","root_shell","su_attempted","is_host_login","is_guest_login"] #not standardizing this as  it is just 0/1

#### 4. Transform all numerical features into a vector using VectorAssembler

In [28]:
assemblerInputs = [ col + "_vector" for col in categoricalColumns ] + numericCols + binaryCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [29]:
stages

[StringIndexer_4dc8ad6fa038118252e4,
 OneHotEncoder_4be6bfbc8a9bf98be0ce,
 StringIndexer_4414bcf864926cc18608,
 OneHotEncoder_4d00bd904a2036c70523,
 StringIndexer_492088f14bcedde14be2,
 OneHotEncoder_4b0ab7f00d516eca2900,
 VectorAssembler_420a8950ba919f70df60]

#### 5. Applying the pipeline stages on the Mixed Union Dataset

In [30]:
# Transformation of the test set
from pyspark.ml import Pipeline
# Create a Pipeline.
pipeline = Pipeline(stages=stages)

# Run the feature transformations.
#  - fit() computes feature statistics as needed.
#  - transform() actually transforms the features.

transformer_test = pipeline.fit(mixedDS)
transformed_df_Mixed = transformer_test.transform(mixedDS)

#### 6. Creating a new column (autoincremental) ID for splitting the Merged Dataset back to Train and Test

In [31]:
from pyspark.sql.functions import monotonically_increasing_id
# This will return a new DF with all the columns + id
transformed_df_Mixed = transformed_df_Mixed.withColumn("id", monotonically_increasing_id())

In [32]:
transformed_df_Mixed.createOrReplaceTempView("Mixed")

In [33]:
transformed_df_Mixed = transformed_df_Mixed.select("features", "label")

#### 7. ReCreating the Training set from the Mixed table view created earlier using SQL

In [34]:
train_new = spark.sql("select * from Mixed Limit 4898431")

In [36]:
train_new.select('protocol_type', 'service', 'dst_bytes', 'connection_type', 'label').limit(10).toPandas()

,protocol_type,service,dst_bytes,connection_type,label
0,tcp,http,45076,normal.,0
1,tcp,http,4528,normal.,0
2,tcp,http,1228,normal.,0
3,tcp,http,2032,normal.,0
4,tcp,http,486,normal.,0
5,tcp,http,1282,normal.,0
6,tcp,http,1337,normal.,0
7,tcp,http,1364,normal.,0
8,tcp,http,1295,normal.,0
9,tcp,http,5450,normal.,0


#### 8. ReCreating the Test set from the Mixed table view created earlier using SQL

In [39]:
test_new = spark.sql("select * from Mixed order by id desc Limit 311029")

In [40]:
test_new.select('protocol_type', 'service', 'dst_bytes', 'connection_type', 'label').limit(10).toPandas()

,protocol_type,service,dst_bytes,connection_type,label
0,udp,private,147,normal.,0
1,udp,private,147,normal.,0
2,udp,private,147,normal.,0
3,udp,private,147,normal.,0
4,udp,private,147,normal.,0
5,udp,private,105,snmpgetattack.,1
6,udp,private,105,normal.,0
7,udp,private,105,snmpgetattack.,1
8,udp,private,105,normal.,0
9,udp,private,105,snmpgetattack.,1


In [41]:
train_new = train_new.select("features", "label")

In [42]:
test_new = test_new.select("features", "label")

## Modelling

#### 1. Caching both training and test data sets

In [43]:
train_new.cache()

DataFrame[features: vector, label: int]

In [44]:
test_new.cache()

DataFrame[features: vector, label: int]

### Splitting the Training Data for Cross Validation

In [45]:
# This takes rather long for some reason (train test split takes long)
### Randomly split data into training (70%) and test (30%) sets. set seed for reproducibility
(train_data, test_data) = train_new.randomSplit([0.7, 0.3])
print('Training records : %d' % train_data.count())
print('Test records : %d ' % test_data.count())
train_data.cache()

Training records : 3427914
Test records : 1470517 


DataFrame[features: vector, label: int]

### Performing a Logistic Regression on CV Train Data (Splitted)

In [46]:
from pyspark.ml.classification import LogisticRegression

# Create initial LogisticRegression model
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10)

# Train model with Training Data
model = lr.fit(train_data)

### Predicting on CV Test Data

In [47]:
predictions = model.transform(test_data)

In [48]:
predictions.limit(5).toPandas()

,features,label,rawPrediction,probability,prediction
0,"(1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[0.271559052145, -0.271559052145]","[0.567475609885, 0.432524390115]",0.0
1,"(1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[0.271559052145, -0.271559052145]","[0.567475609885, 0.432524390115]",0.0
2,"(1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[0.271559052145, -0.271559052145]","[0.567475609885, 0.432524390115]",0.0
3,"(1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[0.271559052145, -0.271559052145]","[0.567475609885, 0.432524390115]",0.0
4,"(1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[0.271559052145, -0.271559052145]","[0.567475609885, 0.432524390115]",0.0


## Evaluation

### 1. Evaluating on CV test data Predictions

In [49]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate the model on the test_data (derived from the original train data)
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
score = evaluator.evaluate(predictions)
print('Score is : %03f' % score )

Score is : 0.999539


### Make predictions on actual test data

In [32]:
predictions = model.transform(test_new)

In [33]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
score = evaluator.evaluate(predictions)
print('Score is : %03f' % score )

Score is : 0.980666


### Cross Validation & Parameter Tuning 

In [34]:
# Cross validate results and fine tune the parameters
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder

evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
grid = ParamGridBuilder(). \
        addGrid(lr.maxIter, [3, 5, 10]). \
        addGrid(lr.regParam, [0.01, 0.001, 0.0001]). \
        addGrid(lr.elasticNetParam, [0.1, 0.5, 1]). \
        build()
cv = CrossValidator(estimator=lr, estimatorParamMaps=grid, evaluator=evaluator)
cvModel = cv.fit(train_data)
cv_score = evaluator.evaluate(cvModel.transform(test_data))

best_model = cvModel.bestModel

In [35]:
print('best score is:' + str(cv_score))
print('best_reg_parameter: ' + str(best_model._java_obj.getRegParam()))
print('best_max_iter: ' + str(best_model._java_obj.getMaxIter()))
print('best_Elastic_Net: ' + str(best_model._java_obj.getElasticNetParam()))

best score is:0.9995476555005792
best_reg_parameter: 0.0001
best_max_iter: 10
best_Elastic_Net: 0.1


### 2. Predicting/Evaluating on test data using the best Model from CV

In [36]:
# Use cross validated parameter tuned on test set
cv_score = evaluator.evaluate(cvModel.transform(test_new))
cv_score

0.9795530436782655